In [1]:
from Crypto.Util.strxor import strxor_c, strxor
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Hash import SHA
import hashlib
import hmac
import math

In [2]:
def hmac_sha1(k_mac_16, m):
    ipad = 0x36
    opad = 0x5C
    k_16bytes = k_mac_16
    k_64bytes_padded = k_16bytes + str('\x00' * (64-len(k_16bytes)))
    #print k_64bytes_padded.encode('hex')
    outer = strxor_c(k_64bytes_padded, opad)
    #print outer.encode('hex')
    inner = strxor_c(k_64bytes_padded, ipad)
    #print inner.encode('hex')
    #hash(outer ∥ hash(inner ∥ message))
    h1 = hashlib.sha1(inner+m).digest()
    #xh1 = SHA.new(inner+m).digest()
    #print h1.encode('hex'), xh1.encode('hex')
    h2 = hashlib.sha1(outer+h1).digest()
    #xh2 = SHA.new(outer+h1).digest()
    #print h2.encode('hex'), xh2.encode('hex')
    return h2

In [3]:
def breakinto_16byte_blocks(inputstring_multipleof_16):
    size = len(inputstring_multipleof_16)
    #print "size", size
    output_list = []
    if size == 16:
        output_list.append(inputstring_multipleof_16)
        #print "no loop"
    else:
        #print "yes loop"
        for i in range(1, size/16+1):
            x = i * 16
            w = (i-1) * 16
            output_list.append(inputstring_multipleof_16[w:x])
    #print output_list
    return output_list
def combinefrom_16byte_chunks(inputlist_16byte_chunks):
    output_string = ""
    for x in inputlist_16byte_chunks:
        output_string+=x
    return output_string

In [4]:
def aes_cbc_enc(k, iv, m):
    m2_blocks = breakinto_16byte_blocks(m)
    #print "m", m.encode("hex")
    #print m2_blocks
    ciphertext_blocks = []
    curr_block_input = ""
    curr_block_iv = iv
    current_block_num = 0
    aes_obj_ecb = AES.new(k, AES.MODE_ECB)
    for block in m2_blocks:
        #print "currently encrypting this block", block.encode("hex")
        curr_block_input = strxor(curr_block_iv, block)
        #alternative way to xor ''.join(chr(ord(a) ^ ord(b)) for a,b in zip(prev_block_output, block))
        curr_block_output = aes_obj_ecb.encrypt(curr_block_input)
        ciphertext_blocks.append(curr_block_output)
        curr_block_iv = curr_block_output
        #print prev_block_output
        current_block_num+=1
    ciphertext = combinefrom_16byte_chunks(ciphertext_blocks)
    return ciphertext

In [5]:
def encrypt(k_enc, k_mac, m):
    hmac1 = hmac_sha1(k_mac, m)
    #print "hmac is", hmac1.encode("hex")
    m1 = m + hmac1
    #print "before padding", m1.encode("hex")
    n = len(m1)%16
    if n != 0:
        ps = bytes(bytearray([16-n]))*(16-n)
    if n == 0:
        ps = '\x10' * 16
    m2 = m1 + ps
    #print "after padding", m2.encode("hex"), "length", len(m2)
    random_generator = Random.new()
    iv = random_generator.read(16)
    #iv = "\x01\x02\x01\x02\x01\x02\x01\x02\x01\x02\x01\x02\x01\x02\x01\x02"
    ciphertext = iv + aes_cbc_enc(k_enc, iv, m2)
    return ciphertext

In [6]:
def aes_cbc_dec(k, iv, c):
    c_blocks = breakinto_16byte_blocks(c)
    #print c.encode("hex")
    #print c_blocks
    plaintext_blocks = []
    curr_block_output = ""
    curr_block_iv = iv
    aes_obj_ecb = AES.new(k, AES.MODE_ECB)
    for block in c_blocks:
        #print "currently decrypting this block", block.encode("hex")
        #print "current block iv", curr_block_iv
        curr_block_output = aes_obj_ecb.decrypt(block)
        plaintext_blocks.append(strxor(curr_block_iv, curr_block_output))
        #alternative way to xor: ''.join(chr(ord(a) ^ ord(b)) for a,b in zip(curr_block_output, curr_block_iv))
        curr_block_iv = block
    plaintext = combinefrom_16byte_chunks(plaintext_blocks)
    return plaintext

In [7]:
def verify_padding(padding):
    size = len(padding)
    for x in padding:
        if x != bytearray([size]):
            return 0
    return 1

In [9]:
def decrypt(k_dec, k_mac, c):
    #print "start"
    #print c
    c = c.decode("hex")
    iv = c[0:16]
    c1 = c[16:]
    #print "iv and c1", iv.encode("hex"), c1.encode("hex")
    m2 = aes_cbc_dec(k_dec, iv, c1)
    print "m2", m2.encode("hex")
    n = int(m2[-1].encode('hex'), 16)
    #print n
    m2_padding = m2[-n:]
    #print m2_padding
    if verify_padding(m2_padding) == 0:
        print "INVALID PADDING"
        return "0"
    m1 = m2[:-n]
    m = m1[:-20]
    t = m1[-20:]
    print m1.encode("hex"), m.encode("hex"), t.encode("hex")
    t1 = hmac_sha1(k_mac, m)
    if t != t1:
        print "INVALID MAC"
        return "0"
    return m

In [10]:
cipt = encrypt("\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00", "gharshchaturvedi", "\x00\x00\x00").encode("hex")
print cipt
plaint = decrypt("\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00", "gharshchaturvedi", cipt)
print plaint

c201dccaaf58e0e2c91eee854780c082960e5fe7c3df1a26b84b67bca2a587fb6a7f3e12d1a997bf845d89bf65d47051
m2 000000dcb4e75b8e3df0410f583c5731908be581c4ea97090909090909090909
000000dcb4e75b8e3df0410f583c5731908be581c4ea97 000000 dcb4e75b8e3df0410f583c5731908be581c4ea97
   


In [59]:
print hmac_sha1("\x0f\x10".encode('hex').decode('hex'), "hello")
h = hmac.new("\x0f\x10", "", hashlib.sha1)
h.update("hello")
print h.hexdigest(), bytes(bytearray(h.hexdigest())), len(h.digest())

0caaa7d8e7078d3ba1602ddb42a1755c1b8dcf4e
0caaa7d8e7078d3ba1602ddb42a1755c1b8dcf4e 0caaa7d8e7078d3ba1602ddb42a1755c1b8dcf4e 20


In [26]:
str2 = "bdddfd4487922193f0f0b83b663572c5487d9a769e52654286072d653ca3caf8b500f0bd02290bbba7b6953d08f1c5f62e574bf2d407c71604ea384c89d917b4"
aes_cbc_dec("1234567890abcdef", "bdddfd4487922193", str2).encode("hex")

'a1e47ad7136a1db096707d7c0e73c3d9bba90888862f1e7e3b6419265f089ed21adfd2f2bd18ed3add16d2aa986b4de59ae7af984a11b992466c0711546fbbca1c4e0e9def6aee0c105b14802d0879b232a1a19bb56099ecf7797de81e232d00ef3f0304281601046df0a835e33fdf5ef42f3e6a4c15e7f0e4d8fa27319e83e7'

In [56]:
verify_padding("abc\x05\x05\x05\x05\x05"[-5:])

1

In [37]:
int(str(bytes(bytearray([15]))*(15))[-1].encode('hex'), 16)

15

In [63]:
kx = "gharshchaturvedi"

In [80]:
hmac_sha1(kx, "\x00\x00\x00")

'dcb4e75b8e3df0410f583c5731908be581c4ea97'

In [71]:
strxor_c("\x11\x11\x11", 0x11)

'\x00\x00\x00'